In [41]:
import sys
import numpy as np
import pandas as pd
sys.path.append('D:/research/jokesclustering/') # ваш путь до корня проекта
from vector_clustering.data.manager import get_jokes_as_dataframe, read_lines_from_file
from gensim.utils import simple_preprocess
from gensim import models, corpora

In [7]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")
from pymorphy2 import MorphAnalyzer
analyzer = MorphAnalyzer()

C:\Users\PC\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
jokes_frame = get_jokes_as_dataframe()
jokes_frame.head()

,joke_text
0,"Мужчину трудно задеть за живое, но уж если зад..."
1,В нашем кемпинге строго запрещено людям разног...
2,А как хорошо у девушек начинается: любимый: ми...
3,"Одна белка случайно попробовала пиво и поняла,..."
4,ОБЪЯВЛЕНИЕ На время мирового финансового кризи...


### Функции предобработки текста
лемматизаторы, чистка стоп-слов, любой необходимый препроцессинг

In [3]:
def gensim_simple_preprocess(text, min_len=3, max_len=1000000):
    ## приводит к нижнему регистру, удаляет знаки препинания, оставляет слова между (min_length, max_length)
    return ' '.join(simple_preprocess(text, min_len=min_len, max_len=max_len))

In [4]:
def rnnmorph_lemmatizer(text):
    ## альтернатива для pymorphy2: приводит слова к нормальной форме.
    return ' '.join([x.normal_form for x in predictor.predict(text.split())])

def pymorphy_lemmatizer(text):
    return ' '.join(analyzer.parse(x)[0].normal_form for x in text.split())

In [5]:
stop_words = set([x.split('\n')[0] for x in read_lines_from_file('ru_stop_words.txt')])

def stop_words_remove(text):
    ## удаление из текста стоп-слов на основании списка стоп-слов
    return ' '.join([x for x in text.lower().split() if x not in stop_words])

In [85]:
### сделать в качестве словаря только Tfidf ?
## если сделать Tfidf, то можно убрать из словаря самые редкие слова и самые частые
### доделать !?

In [8]:
# определяет порядок вызова функций
# rnnmorph_lemmatizer слишком долгий
cleaners = [gensim_simple_preprocess, stop_words_remove, pymorphy_lemmatizer]

def apply_to_text(t):
    for func in cleaners:
        t = func(t)
    return t

def apply_func_to_texts(texts, func):
    return [func(t) for t in texts]

def apply_to_texts(ts):
    return [apply_to_text(t) for t in ts]

def apply_to_texts_func_by_func(texts):
    for func in cleaners:
        texts = apply_func_to_texts(texts, func)
        print('Applied {}'.format(func))
    return texts

In [9]:
%%time
preprocessed_collection = apply_to_texts_func_by_func(jokes_frame.joke_text.values)

Applied <function gensim_simple_preprocess at 0x0000019B31CEB2F0>
Applied <function stop_words_remove at 0x0000019B29AE1048>
Applied <function pymorphy_lemmatizer at 0x0000019B31CEB598>
Wall time: 3min 28s


### Построение lda модели

In [82]:
number_topics = 100

In [15]:
tokenized_collection = [x.split() for x in preprocessed_collection]
dictionary = corpora.Dictionary(tokenized_collection) # словарь вида: dictionary[word] : numeric_index
corpus = [dictionary.doc2bow(text) for text in tokenized_collection] # каждый документ теперь выглядит так:
# [(12, 3), (25, 2), ... ] - то есть в шутке словоа с индексом 12 встретилось 3 раза, слово с индексом 25 - 2 раза и так далее

In [83]:
lda = models.LdaModel(corpus=corpus, 
                      num_topics=number_topics, 
                      id2word=dictionary, 
                      dtype=np.float64)

In [86]:
for idx in range(number_topics):
    print('Topic: {}, words: {}'.format(idx, lda.print_topic(idx, 5)))
    print()

Topic: 0, words: 0.042*"мясо" + 0.035*"помогать" + 0.035*"командировка" + 0.030*"мор" + 0.026*"старик"

Topic: 1, words: 0.206*"давать" + 0.078*"брать" + 0.047*"кот" + 0.042*"попасть" + 0.034*"кухня"

Topic: 2, words: 0.048*"прямо" + 0.047*"кризис" + 0.044*"десять" + 0.040*"общий" + 0.039*"вызывать"

Topic: 3, words: 0.066*"долго" + 0.060*"новое" + 0.055*"сэр" + 0.042*"изя" + 0.036*"похожий"

Topic: 4, words: 0.093*"магазин" + 0.080*"телефон" + 0.057*"никогда" + 0.054*"учительница" + 0.033*"продавец"

Topic: 5, words: 0.213*"другой" + 0.072*"всегда" + 0.048*"рассказать" + 0.040*"сторона" + 0.031*"хотеться"

Topic: 6, words: 0.151*"потом" + 0.070*"курить" + 0.060*"завтра" + 0.058*"выпить" + 0.044*"зарплата"

Topic: 7, words: 0.107*"мальчик" + 0.083*"девочка" + 0.062*"вода" + 0.057*"забыть" + 0.054*"дедушка"

Topic: 8, words: 0.148*"девушка" + 0.135*"день" + 0.117*"или" + 0.062*"парень" + 0.054*"здесь"

Topic: 9, words: 0.200*"хотеть" + 0.123*"делать" + 0.110*"надо" + 0.088*"ничто" + 0.0

Topic: 80, words: 0.117*"никто" + 0.068*"урок" + 0.040*"провести" + 0.040*"янукович" + 0.034*"некоторый"

Topic: 81, words: 0.095*"страна" + 0.076*"неделя" + 0.067*"двое" + 0.045*"язык" + 0.042*"начальник"

Topic: 82, words: 0.085*"хоть" + 0.047*"бежать" + 0.045*"положить" + 0.041*"просыпаться" + 0.038*"счёт"

Topic: 83, words: 0.108*"даже" + 0.094*"понимать" + 0.072*"сосед" + 0.070*"украина" + 0.046*"бросить"

Topic: 84, words: 0.085*"свет" + 0.073*"конец" + 0.046*"оказаться" + 0.045*"закон" + 0.038*"март"

Topic: 85, words: 0.052*"слушай" + 0.046*"директор" + 0.045*"половина" + 0.044*"смысл" + 0.036*"платить"

Topic: 86, words: 0.161*"русский" + 0.084*"пока" + 0.083*"здравствовать" + 0.052*"случай" + 0.035*"продать"

Topic: 87, words: 0.124*"при" + 0.094*"иметь" + 0.057*"часы" + 0.035*"более" + 0.029*"действительно"

Topic: 88, words: 0.091*"два" + 0.078*"разговаривать" + 0.068*"подруга" + 0.049*"люся" + 0.034*"она"

Topic: 89, words: 0.129*"работать" + 0.117*"пойти" + 0.063*"тысяча"

### Присвоение тем шуткам
на выходе получим дата фрейм из одной колонки, эта колонка - самая вероятная тема для данной шутки

In [87]:
most_relevant_topics = [sorted(lda.get_document_topics(bow), key=lambda x: x[1], reverse=True)[0][0] for bow in corpus]

In [88]:
pd.DataFrame({'topic_id' : most_relevant_topics}).to_csv('lda_100_topics.csv')

In [89]:
jokes_frame['topic_id'] = most_relevant_topics

In [90]:
jokes_frame.head()

,joke_text,topic_id
0,"Мужчину трудно задеть за живое, но уж если зад...",65
1,В нашем кемпинге строго запрещено людям разног...,66
2,А как хорошо у девушек начинается: любимый: ми...,96
3,"Одна белка случайно попробовала пиво и поняла,...",48
4,ОБЪЯВЛЕНИЕ На время мирового финансового кризи...,2


### Функции для отображения результатов
теперь тема описывается словами. Чтобы понять, что это за тема и оценить ее качество нужно посмотреть на топ слов и на шутки данной темы

In [91]:
def descirbe_topic(topic_id, top_words=10, top_jokes=10, need_print=True, writing_object=None):
    most_relevant_words = ', '.join(x[0] for x in lda.show_topic(topic_id, topn=top_words))
    sample_jokes = '\n'.join(jokes_frame[jokes_frame.topic_id == topic_id].sample(top_jokes).joke_text.values)
    template = '=====\nTopic {}\nTop words: {}\nSample jokes:\n {}\n=====\n\n'.format(topic_id, most_relevant_words, sample_jokes)
    if need_print:
        print(template)
    if writing_object:
        writing_object.write(template)

In [92]:
with open('lda_100_topics_description.txt', 'w', encoding='utf-8') as f:
    for i in range(number_topics):
        descirbe_topic(topic_id=i, need_print=False, writing_object=f)